# Notebook pour la classification des affiches de cinéma

En classifiant les images, ce Notebook permet de séparer les différents visuels des affiches de cinéma: dessins, photographies ou hybride. Au préalable 3'000 images ont été labellisées selon les trois catégories. Ensuit, différents réseaus de neurones existant et entrainé sont repris pour notre tâche. Les dernières couches de sorties sont enlevées et remplacée par une sortie vide qui permettra la prédiction de nos données.

**Ce Notebook sert d'essaie aux différents modèles mis à jour dans l'état de l'art, pour le colloque Humanistica (Genève, 2022)**

*Plan du Notebook*

> Importer les données

> Charger et appliquer le modèle

> Visualiser les métriques

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
from PIL import ImageFile

%pip install tensorflow_addons
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import Precision, Recall
import tensorflow_addons as tfa
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.models import Model

new_rc_params = {'text.usetex': False,
"svg.fonttype": 'none'
}
plt.rcParams.update(new_rc_params)

## Importer les données

Cette étape demande d'avoir, au préalable, un dossier avec les données d'entrainement et de validation chacun séparé entre les trois catégories.

In [ ]:
# Chemins des données d'entraînement et de validation
train_data_dir = ""
validation_data_dir = ""
# les chemins sont bons?
print(os.path.isdir(train_data_dir), os.path.isdir(validation_data_dir))

### Chargement, transformation et normalisaiton 
 Les arguments définissent une série de transformations aléatoires qui seront appliquées aux images générées pendant l'entraînement.

In [ ]:
# chargement, transformation et normalisaiton 

train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                             shear_range=0.2, 
                                                             zoom_range=0.2, 
                                                             width_shift_range=0.2,
                                                             height_shift_range=0.2,
                                                             horizontal_flip=True, 
                                                             rotation_range=20,
                                                             fill_mode="nearest")
val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

## Chargement du modèle

D'abord les modèles sont importés.  L'entrainement des couches est conservé. Seule la dernière couche est supprimée. Elle est remplacé par une couche vide qui permet de prédire trois catégories. 

Les modèles suivants ont été expérimentés:
- VGG16
- ResNet50
- DenseNet169
- InceptionV3
- MobileNetV2

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import DenseNet169

In [ ]:
# Chargement du modèle pré-entraîné
base_model = MobileNetV2(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3))
#base_model.summary()

### Modification de la fin du modèle pré-entrainé

In [ ]:
# Ajout d'une couche d'agrégation pour réduire les dimensions des données
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
# Ajout d'une couche dense de sortie pour classer les images en trois catégories
predictions = Dense(3, activation='softmax')(x)

In [ ]:
# Congélation des couches du modèle pré-entraîné pour ne pas les entraîner
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Génération du modèle en utilisant les couches précédentes
model = Model(inputs=base_model.input, outputs=predictions)
#model.summary()

### Compilation et entraînement du modèle (+sauvegarde)

Le modèle est ensuite compilé, avec les métriques nécessaires. Finalement il est entrainé puis sauvegardé

In [ ]:
# Compilation du modèle en incluant différentes métriques
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', Precision(), Recall(), F1Score(num_classes=3)])

In [ ]:
# Entraînement du modèle
history = model.fit(
    train_generator,
    steps_per_epoch=74,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=10)

In [ ]:
# la date du jour

from datetime import date
 
today = date.today()
today = str(today).replace("-", "")
today2 = today[-2:] + today[4:-2] + today[2:-4]
print(today2)

In [ ]:
# Sauvegarde du modèle entraîné
nom_du_model = base_model.name+ "_"+ today2
model.save('/models/model_' +nom_du_model+'.h5')

In [ ]:
# Enregistrer les métriques dans un CSV
import pandas as pd

# créer un dataframe pandas avec les mesures
df = pd.DataFrame(history.history)
df.columns = ["loss","accuracy","precision","recall","f1_score","val_loss","val_accuracy","val_precision","val_recall","val_f1_score"]
df["model"] = nom_du_model

# enregistrer le dataframe dans un fichier CSV
df.to_csv(f'/metrics/mesures_{nom_du_model}.csv')

## Visualisations des performances du modèle

Les métriques sont d'abord visualisées et enregistrées séparéement. Dans un second temps, elles sont visualisée ensemble. 

- "Accuracy"
- Perte ("Loss")
- Précision ("Precision")
- Rappel ("Recall")

Pour comprendre, sommairement, les choix de l'algorithme, les mauvaises prédicitions sont visualisées.

In [ ]:
# Accuracy et loss pour l'entrainement et la validation du modèle

fig = plt.figure(figsize=(15,12))

# Plot accuracy
plt.subplot(221)
plt.plot(history.history['accuracy'],'bo--', label = "train")
plt.plot(history.history['val_accuracy'], 'ro--', label = "validation")
plt.title("Accuracy")
plt.ylabel("accuracy")
plt.xlabel("epochs")
plt.legend()

# Plot loss function
plt.subplot(222)
plt.plot(history.history['loss'],'bo--', label = "train")
plt.plot(history.history['val_loss'], 'ro--', label = "validation")
plt.title("Loss")
plt.ylabel("loss")
plt.xlabel("epochs")

plt.legend()
#plt.show()
plt.savefig(f"/content/drive/MyDrive/visualisations/acc_loss_{nom_du_model}.svg")
plt.savefig(f"/content/drive/MyDrive/visualisations/acc_loss_{nom_du_model}.png", dpi=200)

In [ ]:
# Visualisation de l'historique de l'entraînement
precision = history.history['precision']
val_precision = history.history['val_precision']
recall = history.history['recall']
val_recall = history.history['val_recall']

epochs_range = range(10)


plt.figure(figsize=(10, 15))

# Visualisation de la précision
plt.subplot(3, 1, 1)
plt.plot(epochs_range, precision,'o--', color="green",label='Precision entraînement')
plt.plot(epochs_range, val_precision,'o--',color="pink", label='Precision validation')
plt.legend(loc='upper left')
plt.title('Précision')

# Visualisation du recall
plt.subplot(3, 1, 2)
plt.plot(epochs_range , recall, 'o--',color="green", label='Recall entraînement')
plt.plot(epochs_range, val_recall,'o--',color="pink", label='Recall validation')
plt.legend(loc='upper left')
plt.title('Recall')

#plt.show()
plt.savefig(f"/content/drive/MyDrive/visualisations/metriques_{nom_du_model}.svg")
plt.savefig(f"/content/drive/MyDrive/visualisations/metriques_{nom_du_model}.png", dpi=200)

### Prédire les images et visualiser les mauvaises prédictions

In [ ]:
from sklearn.metrics import classification_report

# Prédire les classes pour les images de validation
Y_pred = model.predict_generator(validation_generator, len(validation_generator))

# Convertir les prédictions en étiquettes de classe
y_pred = np.argmax(Y_pred, axis=1)

# Obtenir les noms des classes
class_names = list(train_generator.class_indices.keys())

# Obtenir les étiquettes de classe réelles
y_true = validation_generator.classes

# Afficher le rapport de classification
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
# enregistrer le rapport de classification

report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
dfr = pd.DataFrame(report).transpose()
dfr.to_csv(f'/rapport/rapport_{nom_du_model}.csv')

In [ ]:
# Obtenir les noms des fichiers d'images
filenames = validation_generator.filenames

# Identifier les images mal classées
misclassified_indices = np.where(y_pred != y_true)[0]

# Visualiser les images mal classées
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))

for i, index in enumerate(misclassified_indices[:4]):
    row = i // 2
    col = i % 2
    img = plt.imread(validation_data_dir + '/' + filenames[index])
    axs[row][col].imshow(img)
    axs[row][col].set_title(f"Vraie classe : {class_names[y_true[index]]}\nClasse prédite : {class_names[y_pred[index]]}")
    axs[row][col].axis('off')

plt.tight_layout()
plt.savefig(f"/content/drive/MyDrive/visualisations/misclassified_{nom_du_model}.png", dpi=200)
plt.show()


### Importer et visualiser toutes les métriques dans un graphique

In [ ]:
# Définir les noms des fichiers CSV à charger
directory = '/metrics/'
csv_files = os.listdir(directory)

# Initialiser une liste pour stocker les dataframes pandas chargés à partir des fichiers CSV
dfs = []

# Charger chaque fichier CSV en tant que dataframe pandas et l'ajouter à la liste
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(directory,csv_file))
    dfs.append(df)


# Parcourir chaque dataframe dans la liste et créer un graphique pour chaque métrique
for col in ["loss", "accuracy", "precision", "recall", "f1_score"]:
    fig, ax = plt.subplots(figsize=(10, 6))

    # Parcourir chaque dataframe dans la liste et tracer la courbe de la métrique souhaitée
    for df in dfs:
        ax.plot(df.index, df[col], label=df["model"][0])

    # Définir les étiquettes des axes, le titre et la légende
    ax.set_xlabel("Epoch")
    ax.set_ylabel(col.capitalize())
    ax.set_title(f"{col.capitalize()} over Epochs")
    ax.legend(loc='upper right', prop={'size': 8})

    # Enregistrer le graphique avec le nom de la métrique et la date du jour
    plt.savefig(f"/visualisations/allModels_{col}_{today2}.png")

    # Afficher le graphique
    plt.show()